# QPIXL

In [56]:
# qip/helper.py

import numpy as np

def sfwht(a):
    n = len(a)
    k = ilog2(n)
    j = 1
    while j < n:
        for i in range(n):
            if i & j == 0:
                j1 = i + j
                x = a[i]
                y = a[j1]
                a[i], a[j1] = (x + y) / 2, (x - y) / 2
        j *= 2
    return a            

def isfwht(a):
    n = len(a)
    k = ilog2(n)
    j=1
    while j< n:
        for i in range(n):
            if (i&j) == 0:
                j1=i+j 
                x=a[i]
                y=a[j1]
                a[i],a[j1]=(x+y),(x-y)
        j*=2 
    return a            

def ispow2(x):
    return not (x&x-1)

def nextpow2(x):
    x-=1
    x|=x>>1
    x|=x>>2
    x|=x>>4
    x|=x>>8
    x|=x>>16
    x|=x>>32
    x+=1
    return x 

def ilog2(x):
    return int(np.log2(x))

def grayCode(x):
    return x^(x>>1)

def grayPermutation(a):
    b = np.zeros(len(a))
    for i in range(len(a)):
        b[i] = a[grayCode(i)]
    return b

def invGrayPermutation(a):
    b = np.zeros(len(a))
    for i in range(len(a)):
        b[grayCode(i)] = a[i]
    return b

def convertToAngles(a,maxval):
    pi2=2*np.arctan(1)
    scal = pi2/maxval 
    a = a *scal
    return a

def convertToGrayscale(a,maxval):
    pi2=2*np.arctan(1)
    scal = maxval/pi2 
    a = a * scal
    return a

def countr_zero(n, n_bits=8):
    """Returns the number of consecutive 0 bits in the value of x, starting from the least significant bit ("right")."""
    if n == 0:
        return n_bits

    counts = [0]
    count = 0
    while n != 0:
        if n & 1 == 0:
            count += 1
        elif count > 1:
            counts.append(count)
            count = 0
        n >>= 1
    return max(counts)

In [36]:
grayCode(5) ^ grayCode(6)

2

In [37]:
countr_zero(4)

2

In [38]:
countr_zero(0b00000000)

8

In [39]:
countr_zero(0b11111111)

0

In [40]:
countr_zero(0b00011100)

2

In [41]:
countr_zero(0b00011101)

0

In [12]:
a = [4, -2, 3, -1]
sorted(a, key=abs)

[-1, -2, 3, 4]

In [77]:
# qip/qpixl.py
from qiskit import QuantumCircuit


def cFRQI(a, compression):
    a = convertToAngles(a, 1) # convert grayscale to angles
    
    n = len(a)
    k = ilog2(n)
    
    a = 2*a 
    a = sfwht(a)
    a = grayPermutation(a) 
    
    idx = list(range(0,n))
    a = sorted(a, key=abs)

    # set smallest absolute values of a to zero according to compression param
    cutoff = int((compression / 100.0) * n)
    for it in idx[:cutoff]:
        a[it] = 0.0
    
    # Construct FRQI circuit
    circuit = QuantumCircuit(k + 1)
    # Hadamard register
    circuit.h(range(k))
    # Compressed uniformly controlled rotation register
    ctrl, pc, i = 0, 0, 0
    while i < (1<<k):
        # Reset the parity check
        pc = 0

        # Add RY gate
        if a[i] != 0:
            circuit.ry(a[i], k)

        # Loop over sequence of consecutive zero angles
        while True:
            # Compute control qubit
            if i == (1<<k) - 1:
                ctrl=0
            else:
                ctrl = grayCode(i) ^ grayCode(i + 1)
                ctrl = k - countr_zero(ctrl, n_bits=k) - 1

            # Update parity check
            pc ^= 1 << ctrl
            i += 1
            
            if not (i < (1<<k) and a[i] == 0):
                break

        for j in range(0, k):
            if (pc >> j) & 1:
                circuit.cnot(j, k)
                
    return circuit

In [78]:
a = np.random.randint(0, high=256, size=2**9)
compression = 0

In [79]:
circuit = cFRQI(a, compression)

In [80]:
circuit.draw()

┌───┐                                                               »
q_0: ──┤ H ├───────────────────────────────────────────────────────────────»
       ├───┤                                                               »
q_1: ──┤ H ├───────────────────────────────────────────────────────────────»
       ├───┤                                                               »
q_2: ──┤ H ├───────────────────────────────────────────────────────────────»
       ├───┤                                                               »
q_3: ──┤ H ├───────────────────────────────────────────────────────────────»
       ├───┤                                                               »
q_4: ──┤ H ├───────────────────────────────────────────────────────────────»
       ├───┤                                                               »
q_5: ──┤ H ├───────────────────────────────────────────────────────────────»
       ├───┤                                                               »
q_6: ──┤ H ├────────────────────────────────────────────────────────────■──»
       ├───┤                                                            │  »
q_7: ──┤ H ├────────────────────────────────────────────────────────────┼──»
       ├───┤                                                            │  »
q_8: ──┤ H ├────■──────────────────■─────────────────■──────────────────┼──»
     ┌─┴───┴─┐┌─┴─┐┌────────────┐┌─┴─┐┌───────────┐┌─┴─┐┌────────────┐┌─┴─┐»
q_9: ┤ Ry(0) ├┤ X ├┤ Ry(-π/256) ├┤ X ├┤ Ry(π/256) ├┤ X ├┤ Ry(-π/256) ├┤ X ├»
     └───────┘└───┘└────────────┘└───┘└───────────┘└───┘└────────────┘└───┘»
«                                                                       »
«q_0: ──────────────────────────────────────────────────────────────────»
«                                                                       »
«q_1: ──────────────────────────────────────────────────────────────────»
«                                                                       »
«q_2: ──────────────────────────────────────────────────────────────────»
«                                                                       »
«q_3: ──────────────────────────────────────────────────────────────────»
«                                                                       »
«q_4: ──────────────────────────────────────────────────────────────────»
«                                                                       »
«q_5: ──────────────────────────────────────────────────────────────────»
«                                                                       »
«q_6: ──────────────────────────────────────────────────────────────────»
«                                                                       »
«q_7: ──────────────────────────────────────────────────────────────────»
«                                                                       »
«q_8: ───────────────────■─────────────────────■─────────────────────■──»
«     ┌───────────────┐┌─┴─┐┌───────────────┐┌─┴─┐┌───────────────┐┌─┴─┐»
«q_9: ┤ Ry(-0.036816) ├┤ X ├┤ Ry(-0.073631) ├┤ X ├┤ Ry(-0.085903) ├┤ X ├»
«     └───────────────┘└───┘└───────────────┘└───┘└───────────────┘└───┘»
«                                                                  »
«q_0: ─────────────────────────────────────────────────────────────»
«                                                                  »
«q_1: ─────────────────────────────────────────────────────────────»
«                                                                  »
«q_2: ─────────────────────────────────────────────────────────────»
«                                                                  »
«q_3: ─────────────────────────────────────────────────────────────»
«                                                                  »
«q_4: ─────────────────────────────────────────────────────────────»
«                                                                  »
«q_5: ─────────────────■───────────────────────────────────────────»
«       